In [ ]:
import numpy as np
import json

In [ ]:
pokr = pd.read_json('./pokr.json')
member = pd.read_json('./member.json')

In [ ]:
pokr = pokr[['content','person_id','meeting_id']]
member = member[['person_id','party']]
member['person_id'] = member['person_id'].astype(float)
merge_data = pd.merge(pokr, member, how = 'left', on = 'person_id')

In [ ]:
meet_dic = {}

# meet_dic에 회의별로 발언, 발언자, 정당 저장

In [ ]:
for row in merge_data.iterrows():
    content = row[1]['content']
    p_id = row[1]['person_id']
    party = row[1]['party']
    meet_id = row[1]['meeting_id']
    
    real_content = (re.sub('[=……ㆍ◎,◯!―()\n]', '', content))
   
    meet_dic.setdefault(meet_id,[])
    
    meet_dic[meet_id].append([real_content,p_id,party])

In [ ]:
with open('meet_dic.json', 'w') as fp:
    json.dump(meet_dic, fp)

In [ ]:
with open('meet_dic.json', 'r') as fp:
    meet_dic = json.load(fp)

# con은 보수, pro는 진보, all은 전체

In [ ]:
con_Q = []
con_A = []

pro_Q = []
pro_A = []

pro_Q_plus = []
pro_A_plus = []

con_Q_plus = []
con_A_plus = []

In [ ]:
progress = set([1,2,3,4]) #진보 1,2,3,4
conser = set([0,5,6]) #보수 0,5,6

# 발언 길이가 200이하, 속한 정당이 있어야하고(nan은 장관 등.) 발언자가 위원장이 아니면 추가

In [ ]:
for key in meet_dic.keys():
    Q_pro = []
    A_pro = []
    
    Q_con = []
    A_con = []
    lead_id = meet_dic[key][0][1]
    
    for i in range(1,len(meet_dic[key])-2,2):
        comment = meet_dic[key][i][0]
        person_id = meet_dic[key][i][1]
        party = meet_dic[key][i][2]
        
        next_comment = meet_dic[key][i+1][0]
        next_person_id = meet_dic[key][i+1][1]
        next_party = meet_dic[key][i+1][2]
        
        if person_id != lead_id and next_person_id != lead_id:
            if len(comment) <=200 and len(comment) >0 and len(next_comment)>0 and len(next_comment) <= 200:
                if np.isnan(party) == False and np.isnan(next_party) ==False :
                    if next_party in progress:
                        Q_pro.append([comment,person_id])
                        A_pro.append([next_comment,next_person_id])
                    
                    if next_party in conser:
                        Q_con.append([comment,person_id])
                        A_con.append([next_comment,next_person_id])
    
    pro_Q.extend(Q_pro)
    pro_A.extend(A_pro)
    con_Q.extend(Q_con)
    con_A.extend(A_con)
    
    pro_Q_plus.extend(A_con[:-1])
    pro_A_plus.extend(Q_con[1:])
    con_Q_plus.extend(A_pro[:-1])
    con_A_plus.extend(Q_pro[1:])
    

In [ ]:
con_Q = con_Q + con_Q_plus
con_A = con_A + con_A_plus

pro_Q = pro_Q + pro_Q_plus
pro_A = pro_A + pro_A_plus

In [ ]:
with open('con_Q.json', 'w') as fp:
    json.dump(con_Q, fp)
with open('con_A.json', 'w') as fp:
    json.dump(con_A, fp)
with open('pro_Q.json', 'w') as fp:
    json.dump(pro_Q, fp)
with open('pro_A.json', 'w') as fp:
    json.dump(pro_A, fp)

In [ ]:
all_Q = con_Q + pro_Q
all_A = con_A + pro_A

In [ ]:
with open('all_Q.json', 'w') as fp:
    json.dump(all_Q, fp)
with open('all_A.json', 'w') as fp:
    json.dump(all_A, fp)